# Photoreceptor segmentation using [napari](https://napari.org/) and [cellpose](https://cellpose-napari.readthedocs.io/en/latest/index.html)

## This notebook still has some junk code

### Basic usage:
```python
viewer = napari.Viewer()
nbscreenshot(viewer)
viewer.close()
```

example notebooks [here](https://github.com/sofroniewn/napari-training-course/blob/master/lessons/)

In [ ]:
## to install
# !pip install napari[all]
# !pip install QT
# !pip install napari[pyqt5]
# !pip install cellpose-napari==0.1.4
# !pip install napari-nikon-nd2
# !pip install magicgui
# !pip install napari-pyclesperanto-assistant
# !pip install napari-clusters-plotter #hdbscan failed
# !pip install napari-plot-profile
# !pip install napari-brightness-contrast
# !pip install napari-curtain
# !pip install napari-3d-ortho-viewer
# !pip install napari-manual-split-and-merge-labels
# !pip install napari-crop
# !pip install napari-stl-exporter

## to upgrade:
# !pip install cellpose-napari==0.1.4
# !pip install cellpose-napari --upgrade
# !pip install cellpose --upgrade
# !pip install napari --upgrade
# !pip install mxnet-mkl

In [11]:
import napari
import cellpose_napari
import cellpose
from cellpose import models
from napari.utils import nbscreenshot
from tifffile import imread
import numpy as np
from scipy import ndimage
from scipy.stats import mannwhitneyu
import napari_nikon_nd2
import os 
from magicgui import magicgui
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
import importlib
import svgutils
from svgutils.compose import *
from cmcrameri import cm #colormaps

In [6]:
plotStyle = 'dark';
# plotStyle = 'light';


batlow = cm.vik(range(255))
batlow = batlow[0:255:int(np.floor(255/8)),:]
matplotlib.rcParams['axes.prop_cycle'] = matplotlib.cycler(color=batlow) 

#gene Colors
zfC = {
    'rho'  : '#7d7d7d',
    'sws1' : '#B73AB9',
    'sws2' : '#4364F6',
    'mws1' : '#59CB3B',
    'mws2' : '#59CB3B',
    'mws3' : '#59CB3B',
    'mws4' : '#59CB3B',
    'lws1' : '#CE2A22',
    'lws2' : '#CE2A22',
    'actb2': '#926645',
    'tbx2a': '#c92675',
    'tbx2b': '#7526c9',
    'six7' : '#d6ab00',
}

zfG = {
    'wt' : '#000000',
    'tbx2a' : '#ab266b',
    'tbx2b' : '#421f8e',
    'foxq2' : '#001dd6',
    'skor1a' : '#5456A5',
    'lrrfip1a': '#c7ff5a',
    'xbp1': '#B73AB9',
}

zfGm = {
    'wt' : 'o',
    'tbx2a' : 'P',
    'tbx2b' : 'X',
    'foxq2' : '^',
    'skor1a' : 'h',
    'lrrfip1a' : 'h',
    'xbp1' : 'h',
}

prLabel = {
    'R'  : 'R',
    'U' : 'UV',
    'S' : 'S',
    'M' : 'M',
    'L' : 'L',
}

if plotStyle=='dark':
    # dark background
    params = {"ytick.color" : "w",
              "xtick.color" : "w",
              "axes.labelcolor" : "w",
              "axes.edgecolor" : "w",
             "axes.linewidth" : 3,
             "xtick.major.width" : 3,
             "ytick.major.width" : 3,
             "xtick.major.size" : 8,
             "ytick.major.size" : 8,
             "text.color" : "w"}
    plt.rcParams.update(params)
elif plotStyle=='light':
    # white background
    params = {"ytick.color" : "k",
              "xtick.color" : "k",
              "axes.labelcolor" : "k",
              "axes.edgecolor" : "k",
             "axes.linewidth" : 3,
             "xtick.major.width" : 3,
             "ytick.major.width" : 3,
             "xtick.major.size" : 8,
             "ytick.major.size" : 8,
             "text.color" : "k"}
    plt.rcParams.update(params)
font_prop = font_manager.FontProperties(fname='/System/Library/Fonts/Avenir.ttc')


def formatFigureMain(figH, axH, plotH):
    font_path = '/System/Library/Fonts/Avenir.ttc'
    fontTicks = font_manager.FontProperties(fname=font_path, size=30)
    fontLabels = font_manager.FontProperties(fname=font_path, size=36)
    fontTitle = font_manager.FontProperties(fname=font_path, size=32)
    axH.set_xscale('linear')
    axH.spines['top'].set_visible(False)
    axH.spines['right'].set_visible(False)
    
    for label in (axH.get_xticklabels() + axH.get_yticklabels()):
        label.set_fontproperties(fontTicks)
    axH.set_xlabel(axH.get_xlabel(), fontproperties = fontTicks)
    axH.set_ylabel(axH.get_ylabel(), fontproperties = fontTicks)
    return fontLabels

def lighten_color(color, amount=0.5):
    """
    Lightens the given color by multiplying (1-luminosity) by the given amount.
    Input can be matplotlib color string, hex string, or RGB tuple.

    Examples:
    >> lighten_color('g', 0.3)
    >> lighten_color('#F034A3', 0.6)
    >> lighten_color((.3,.55,.1), 0.5)
    """
    import colorsys
    try:
        c = matplotlib.colors.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*matplotlib.colors.to_rgb(c))
    return matplotlib.colors.rgb2hex(colorsys.hls_to_rgb(c[0], 1 - amount * (1 - c[1]), c[2]))

def estimateJitter(dataArray):
    """ creates random jitter scaled by local density of points"""
    from scipy.stats import gaussian_kde
    kde = gaussian_kde(dataArray)
    density = kde(dataArray)
    jitter = np.random.randn(len(dataArray))*density
    return jitter

# viewer = napari.Viewer()

print('Opened viewer; plot style is ' + plotStyle)

Opened viewer; plot style is dark


In [7]:
viewer = napari.Viewer()

2022-01-11 14:20:48,194 [INFO] Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2022-01-11 14:20:48,195 [INFO] NumExpr defaulting to 8 threads.


[INFO] Resource 'XMLSchema.xsd' is already loaded


2022-01-11 14:20:49,373 [INFO] Resource 'XMLSchema.xsd' is already loaded


In [ ]:
# viewer.close()

***
# Index <a id='Index'></a>
***
- [Extraction of z planes for analysis](#zExtract)
- [Segmentation with cellpose](#cellSeg)
- [Manual correction of segmentation](#manualCuration)
- [Create thumbnails _WIP_](#thumbnails)
- [Quantification](#quantification)

***
## Extract layers from z-stacks<a id='zExtract'>∮</a>
***
[Back to Index](#Index)

In [ ]:
# getting list of files
dPath = "/Volumes/angueyraNEI/LiImaging/zf/20211129_skor1aF0_xOG_aCtdT/"
!ls $dPath/*[002,004].nd2

In [ ]:
# clear viewer
viewer.layers.select_all()
viewer.layers.remove_selected()

In [ ]:
# clear viewer
viewer.layers.select_all()
viewer.layers.remove_selected()

# analysis directory
dAnalysis = "/Users/angueyraaristjm/Documents/LiImaging/Analysis/CRskor1aF0s/xOGaCT/"

# open file

# uninjected
# dPath = "/Volumes/angueyraNEI/LiImaging/zf/20211129_skor1aF0_xOG_aCtdT/" # bad imaging: did not configure A1R properly, so there's bleedthrough from GFP into RFP channel
# fPath = "20211129_5dpf_xOG_aCtdT_wt_L07_002"; gene = 'wt'; zlims = [31,33];

# F0[skor1a]
dPath = "/Volumes/angueyraNEI/LiImaging/zf/20211129_skor1aF0_xOG_aCtdT/" # bad imaging: did not configure A1R properly, so there's bleedthrough from GFP into RFP channel
# fPath = "20211129_5dpf_xOG_aCtdT_skor1aF0_L07_002"; gene = 'skor1a'; zlims = [29,30];


# create subdirectory for analysis
dOut = dAnalysis + gene + '/' + fPath + '/'
if (os.path.isdir(dOut)==False):
    os.mkdir(dOut)
    print('Created output directory')

print('Canvas clear')

In [ ]:
# load whole stack to figure out best planes for cellpose
viewer.open((dPath+fPath + ".nd2"))
viewer.layers.remove(viewer.layers[2]) # remove transmitted detector image
viewer.layers[0].colormap = 'green'
viewer.layers[0].name = 'R'
viewer.layers[1].colormap = 'magenta'
viewer.layers[1].name = 'C'
viewer.layers.select_next();
print('Loaded!')

In [ ]:
# make mips (and remove any previous ones)
# # code for eyes without immuno
zlims = [30,31];
if len(viewer.layers)>2:
    for l in viewer.layers[2:]:
        viewer.layers.remove(l)

for l in [viewer.layers[0],viewer.layers[1]]:
    l.visible = False
#     viewer.layers.remove(l.name + '_mip')
    viewer.add_image(l.data[zlims[0]:zlims[1]].max(axis=0), blending='additive', colormap = l.colormap, name = l.name + "_mip")
    
    
# # for FPs ------- # for OSs # code for eyes with 1D4 immuno
# zlims = [28,30]; zlims2 = [14,18];
# if len(viewer.layers)>3:
#     for l in viewer.layers[3:]:
#         viewer.layers.remove(l)

# for l in [viewer.layers[0],viewer.layers[1]]:
#     l.visible = False
# #     viewer.layers.remove(l.name + '_mip')
#     viewer.add_image(l.data[zlims[0]:zlims[1]].max(axis=0), blending='additive', colormap = l.colormap, name = l.name + "_mip")
    
# for l in [viewer.layers[2]]:
#     l.visible = False
# #     viewer.layers.remove(l.name + '_mip')
#     viewer.add_image(l.data[zlims2[0]:zlims2[1]].max(axis=0), blending='additive', colormap = l.colormap, name = l.name + "_mip")

In [ ]:
# when things look good, save in folder for batch cellpose analysis
l = viewer.layers['C_mip']; l.save(dOut + l.name + '.tiff')
l = viewer.layers['R_mip']; l.save(dOut + l.name + '.tiff')
print('MIP layers saved')
# l = viewer.layers['LWS_mip']; l.save(dOut + l.name + '.tiff')

### [Go back and run next one &uarr;](#zExtract)

***
## Segmentation using cellpose<a id='cellSeg'>∮</a>
***
[Back to Index](#Index)

## Run cellpose in napari GUI and identify good parameters.
Usually for:
1. xOPS:GFP: diameter = 17; flow_threshold = 0.4 and mask_threshold = 0.0
1. sws1:mCherry:
1. sws2:mCherry:
1. mws2:GFP:
1. thrb:tdTomato:
    

In [ ]:
# clear viewer
viewer.layers.select_all()
viewer.layers.remove_selected()

# define analysis directory
dAnalysis = "/Users/angueyraaristjm/Documents/LiImaging/Analysis/CRskor1aF0s/xOGaCT/"

# open example file
# uninjected
dPath = "/Volumes/angueyraNEI/LiImaging/zf/20211129_skor1aF0_xOG_aCtdT/" # bad imaging: did not configure A1R properly, so there's bleedthrough from GFP into RFP channel
fPath = "20211129_5dpf_xOG_aCtdT_wt_L07_002"; gene = 'wt';

# redefine analysis subdirectory
dOut = dAnalysis + gene + '/' + fPath + '/'
print('Viewer cleared...')

## Cellpose: batch analysis
#### Define parameters and files

In [15]:
# define analysis directory
dAnalysis = "/Users/angueyraaristjm/Documents/LiImaging/Analysis/CRtest/"

# collect file paths
gene = 'wt'
fPaths = [
#     "20211129_5dpf_xOG_aCtdT_wt_L07_002",
#     "20211129_5dpf_xOG_aCtdT_wt_L07_004",
    "20211129_5dpf_xOG_aCtdT_wt_L09_002",
    "20211129_5dpf_xOG_aCtdT_wt_L09_004",
         ]

# define cellpose params
cpParams = {
    'model' : 'cyto2', # default is 'cyto' or 'cyto2'
    'net_avg' : False,
    'channels' : [0,0], #single channel without nucleaus info
    'diameter' : 17,
    'flow_threshold' : 0.4,
    'mask_threshold' : 0.0
}

# define model to use (e.g. 'cyto2')
model = models.Cellpose(gpu=False, model_type=cpParams['model'])

2022-01-11 14:23:29,508 [INFO] >>>> using CPU


#### Run in loop to track progress and automatically save results
> In Jan 2022: ~4 minutes per 1024 x 1024 image (4 nets). 

In [18]:
print('Starting analysis:')
for fPath in fPaths:
    dOut = dAnalysis + gene + '/' + fPath + '/' + 'R_mip.tiff'
    print('\t' + dOut)
    img = cellpose.io.imread(dOut)
#     imgplot = plt.imshow(img)
    masks, flows, styles, diams = model.eval(img, 
                                             diameter=cpParams['diameter'], channels=cpParams['channels'],
                                             do_3D=False, net_avg = cpParams['net_avg'], interp = True,
                                             flow_threshold = cpParams['flow_threshold'], mask_threshold = cpParams['mask_threshold'])
    cellpose.io.masks_flows_to_seg(img, masks, flows, diams, dOut, cpParams['channels']) # save results 
    
print('Finished cellpose batch analysis')

Starting analysis:
	/Users/angueyraaristjm/Documents/LiImaging/Analysis/CRtest/wt/20211129_5dpf_xOG_aCtdT_wt_L09_002/R_mip.tiff
2022-01-11 14:27:30,525 [INFO] ~~~ FINDING MASKS ~~~
2022-01-11 14:28:30,706 [INFO] >>>> TOTAL TIME 60.18 sec
	/Users/angueyraaristjm/Documents/LiImaging/Analysis/CRtest/wt/20211129_5dpf_xOG_aCtdT_wt_L09_004/R_mip.tiff
2022-01-11 14:28:31,017 [INFO] ~~~ FINDING MASKS ~~~
2022-01-11 14:29:31,581 [INFO] >>>> TOTAL TIME 60.56 sec
Finished cellpose batch analysis


### Old code: running cellpose through cellpose-napari

In [ ]:
# # after running cellpose, rename cellpose images and save before manual curation:
# baseName = 'R'
# lname = baseName + '_mip'
# viewer.layers[lname + '_cp_masks_000'].name = baseName + '_seg'
# viewer.layers[lname + '_cp_outlines_000'].name = baseName + '_outlines'
# viewer.layers[lname + '_cp_cellprob_000'].name = baseName + '_prob'
# viewer.layers[lname + '_cp_flows_000'].name = baseName + '_flows'

# l = viewer.layers[baseName + '_seg']; l.save(dOut + l.name + '.tiff')
# l = viewer.layers[baseName + '_outlines']; l.save(dOut + l.name + '.tiff')
# l = viewer.layers[baseName + '_prob']; l.save(dOut + l.name + '.tiff')
# l = viewer.layers[baseName + '_flows']; l.save(dOut + l.name + '.tiff')

In [ ]:
# # after running cellpose, rename cellpose images and save before manual curation:
# baseName = 'C'
# lname = baseName + '_mip'
# viewer.layers[lname + '_cp_masks_000'].name = baseName + '_seg'
# viewer.layers[lname + '_cp_outlines_000'].name = baseName + '_outlines'
# viewer.layers[lname + '_cp_cellprob_000'].name = baseName + '_prob'
# viewer.layers[lname + '_cp_flows_000'].name = baseName + '_flows'

# l = viewer.layers[baseName + '_seg']; l.save(dOut + l.name + '.tiff')
# l = viewer.layers[baseName + '_outlines']; l.save(dOut + l.name + '.tiff')
# l = viewer.layers[baseName + '_prob']; l.save(dOut + l.name + '.tiff')
# l = viewer.layers[baseName + '_flows']; l.save(dOut + l.name + '.tiff')

***
## Manual correction of cellpose segmentation<a id='manualCuration'>∮</a>
***
[Back to Index](#Index)

In [ ]:
# clear viewer
viewer.layers.select_all()
viewer.layers.remove_selected()


# analysis directory
dAnalysis = "/Users/angueyraaristjm/Documents/LiImaging/Analysis/CRtest/"

# open file
# uninjected
dPath = "/Volumes/angueyraNEI/LiImaging/zf/20211129_skor1aF0_xOG_aCtdT/" # bad imaging: did not configure A1R properly, so there's bleedthrough from GFP into RFP channel
# fPath = "20211129_5dpf_xOG_aCtdT_wt_L07_002"; gene = 'wt'; zlims = [31,33];
# fPath = "20211129_5dpf_xOG_aCtdT_wt_L07_004"; gene = 'wt'; zlims = [28,30];
fPath = "20211129_5dpf_xOG_aCtdT_wt_L09_002"; gene = 'wt'; zlims = [37,38];
# fPath = "20211129_5dpf_xOG_aCtdT_wt_L09_004"; gene = 'wt'; zlims = [25,26]; # weird one; almost no rods...


# define subdirectory for analysis
dOut = dAnalysis + gene + '/' + fPath + '/'
print('Viewer cleared...')

# clear key binds
@viewer.bind_key('k', overwrite=True)
def toggle_sel(viewer):
    ...
@viewer.bind_key('b', overwrite=True)
def toggle_sel(viewer):
    ...
@viewer.bind_key('Shift-x', overwrite=True)
def removeLabel(viewer):
    ...

# load mips
viewer.open(dOut + 'R' + "_mip.tiff", plugin='builtins', colormap = 'green', blending='additive');
# viewer.open(dOut + 'C' + "_mip.tiff", plugin='builtins', colormap = 'magenta', blending='additive');
# viewer.layers['C_mip'].visible = True

# load segmentation
segData = np.load(dOut + 'R' + "_mip" + "_seg.npy", allow_pickle=True).item()
viewer.add_labels(segData['masks'], name='R_seg',blending='additive');
# viewer.layers['R_seg'].preserve_labels = True;

# segData = np.load(dOut + 'C' + "_mip" + "_seg.npy", allow_pickle=True).item()
# viewer.add_labels(segData['masks'], name='C_seg',blending='additive');
# viewer.layers['C_seg'].visible = False
# viewer.layers['C_seg'].preserve_labels = True;

# viewer.layers['R_seg'].contour = 5
# viewer.layers['C_seg'].contour = 5

print('Loaded  ' + fPath + ' !\n Now fix it and save it!')

#define useful keyboard shortcuts

@viewer.bind_key('Shift-x', overwrite=True)
def removeLabel(viewer):
    lname = 'R_seg'
    tempd = viewer.layers[lname].data
    tempd[tempd == viewer.layers[lname].selected_label]=0
    viewer.layers[lname].data = tempd
    print('Cut!')

print('removeLabel on R_seg ("Shift-X")')

@viewer.bind_key('k', overwrite=True)
def toggle_sel(viewer):
    lname = 'R_seg'
    if (viewer.layers[lname].preserve_labels == True):
        viewer.layers[lname].preserve_labels = False
    elif (viewer.layers[lname].preserve_labels == False):
        viewer.layers[lname].preserve_labels = True
print('toggle R_seg preserve_labels ("k")')
        
@viewer.bind_key('b', overwrite=True)
def toggle_sel(viewer):
    lname = 'C_mip'
    if (viewer.layers[lname].visible == True):
        viewer.layers[lname].visible = False
    elif (viewer.layers[lname].visible == False):
        viewer.layers[lname].visible = True
print('toggle C_mip visibility ("B")')

@viewer.bind_key('n', overwrite=True)
def toggle_sel(viewer):
    lname = 'R_mip'
    if (viewer.layers[lname].visible == True):
        viewer.layers[lname].visible = False
    elif (viewer.layers[lname].visible == False):
        viewer.layers[lname].visible = True
print('toggle R_mip visibility ("N")')

In [ ]:
# load z-stack if needed
# dPath = "/Volumes/zfSSD/LiImaging/A1R/zf/20201030_CRtbx2ab/"
# viewer.open((dPath+fPath + ".nd2"))
# viewer.layers.remove(viewer.layers[fPath+' [2]']) # remove transmitted detector image
# viewer.layers.remove(viewer.layers[fPath+' [3]']) # remove immuno layer
# viewer.layers[fPath].colormap = 'magenta'
# viewer.layers[fPath].name = 'Mz'
# viewer.layers[fPath+' [1]'].colormap = 'green'
# viewer.layers[fPath+' [1]'].visible = False
# viewer.layers[fPath+' [1]'].name = 'Lz'
# viewer.layers.select_next();
# print('Loaded!')

#### resave curated segmentation after napari-ing around (only doing R_mip)

In [ ]:
baseName = 'R'
lname = baseName + '_seg'
l = viewer.layers[baseName + '_seg']; l.save(dOut + l.name + '_curated.tiff')
print('Done with ' + lname + ' for ' + fPath)

### [Go back and run next one &uarr;](#manualCuration)

In [ ]:
# manual merging of labels
mergeLabels = [167,168]
# lname = 'M_seg'
lname = 'L_seg'
if viewer.layers[lname].visible == False:
    print('Layer not active')
else:
    tempd = viewer.layers[lname].data
    tempd[tempd == mergeLabels[1]]=mergeLabels[0]
    viewer.layers[lname].data = tempd
    print('Merged in ' + lname)

In [ ]:
# manually delete label
selLabel = 49
lname = 'L_seg'
tempd = viewer.layers[lname].data
tempd[tempd == selLabel]=0
viewer.layers[lname].data = tempd

#### resave curated segmentation after napari-ing around

In [ ]:
baseName = 'R'
lname = baseName + '_seg'
l = viewer.layers[baseName + '_seg']; l.save(dOut + l.name + '_curated_incomplete.tiff')
# l = viewer.layers[baseName + '_seg']; l.save(dOut + l.name + '_curated.tiff')
viewer.layers['L_mip'].visible = True
viewer.layers['L_seg'].visible = True
viewer.layers['M_seg'].contour = 0
viewer.layers['M_mip'].visible = False
viewer.layers['M_seg'].visible = False
print('Done with ' + lname)

@viewer.bind_key('Shift-x', overwrite=True)
def removeLabel(viewer):
#     lname = 'M_seg'mvvvv
    lname = 'L_seg'
    tempd = viewer.layers[lname].data
    tempd[tempd == viewer.layers[lname].selected_label]=0
    viewer.layers[lname].data = tempd
    print('Cut!')

print('removeLabel on L_seg ("Shift-X")')

@viewer.bind_key('k', overwrite=True)
def toggle_sel(viewer):
    lname = 'L_seg'
    if (viewer.layers[lname].preserve_labels == True):
        viewer.layers[lname].preserve_labels = False
    elif (viewer.layers[lname].preserve_labels == False):
        viewer.layers[lname].preserve_labels = True
print('toggle L_seg preserve_labels ("k")')

@viewer.bind_key('b', overwrite=True)
def toggle_sel(viewer):
    lname = 'M_mip'
    if (viewer.layers[lname].visible == True):
        viewer.layers[lname].visible = False
    elif (viewer.layers[lname].visible == False):
        viewer.layers[lname].visible = True
print('toggle L_mip visibility ("B")')

        
@viewer.bind_key('n', overwrite=True)
def toggle_sel(viewer):
    lname = 'L_mip'
    if (viewer.layers[lname].visible == True):
        viewer.layers[lname].visible = False
    elif (viewer.layers[lname].visible == False):
        viewer.layers[lname].visible = True

print('toggle L_mip visibility ("N")')

#### resave curated segmentation after napari-ing around

In [ ]:
baseName = 'L'
lname = baseName + '_seg'
l = viewer.layers[baseName + '_seg']; l.save(dOut + l.name + '_curated.tiff')
print('Done with ' + lname)

In [ ]:
# baseName = 'M'
# l = viewer.layers[baseName + '_seg']; l.save(dOut + l.name + '_curated.tiff')

### Reload curated segmentations after saving

In [ ]:
# remove all except mips
if len(viewer.layers)>2:
    for l in viewer.layers[2:]:
        viewer.layers.remove(l)

In [ ]:
# viewer.open(dOut + 'M' + "_seg_curated_incomplete.tiff", name='M_seg', plugin='builtins');
viewer.open(dOut + 'R' + "_seg_curated.tiff", name='R_seg', plugin='builtins');
viewer.layers['R_seg'].contour = 5

print(fPath)
nR = len(np.unique(viewer.layers['R_seg'].data))-1
print("Image has {0} Rods".format(nR))
# nR = len(np.unique(viewer.layers['L_seg'].data))-1
# print("Image has {0} L cones".format(nR))

In [ ]:
# remove all layers
if len(viewer.layers)>0:
    for l in viewer.layers[0:]:
        viewer.layers.remove(l)

#### 

***
## To Do: Batch export of png with mips and contours of curated segmentation<a id='thumbnails'>∮</a>
***
[Back to Index](#Index)

This could do the trick:
```python
def blended_img(viewer):
    import napari
    import numpy as np
    
    blended = np.zeros(viewer.layers[0].data.shape + (4,))
    for layer in viewer.layers:
        # normalize data by clims
        normalized_data = (layer.data - layer.contrast_limits[0]) / (
        layer.contrast_limits[1] - layer.contrast_limits[0]
    )
        colormapped_data = layer.colormap.map(normalized_data.flatten())
        colormapped_data = colormapped_data.reshape(normalized_data.shape + (4,))

        blended = blended + colormapped_data
    
    blended[..., 3] = 1 # set alpha channel to 1

    return blended
```

***
## Quantification<a id='quantification'></a>
***
[Back to Index](#Index)

### Compile counts by looping through all files

In [ ]:
def cellCounter(viewer,dAnalysis,gene,fPath,photoLabel):
    import os.path
    # clear viewer
    for l in viewer.layers:
        viewer.layers.remove(l)
    viewer.layers.select_all()
    viewer.layers.remove_selected()
    dOut = dAnalysis + gene + '/' + fPath + '/'
    # load segmentation
    segPath = dOut + photoLabel + "_seg_curated.tiff"
    if os.path.isfile(segPath):
        viewer.open(segPath, name='Seg', plugin='builtins', blending='additive');
        nCells = np.unique(viewer.layers['Seg'].data).shape[0]
    else:
        nCells = float("nan")
    print(fPath + ': nR = ' + str(nCells)) 
#     np.savez(dOut + 'quantificationGFP.npz',
#              lcones=lcones,
#              lRFP=lRFP,
#              lGFP=lGFP,
#              lRFPsd=lRFPsd,
#              lGFPsd=lGFPsd,
#              lRtile=lRtile,
#              lGtile=lGtile)
#     results = zip(['lcones','lRFP','lGFP','lRFPsd','lGFPsd','lRtile','lGtile'],
#                  [lcones,lRFP,lGFP,lRFPsd,lGFPsd,lRtile,lGtile])
    
    return nCells

In [ ]:
# because it's simple, just going to create csv manually
# analysis directory
dAnalysis = "/Users/angueyraaristjm/Documents/LiImaging/Analysis/CRskor1aF0s/xOGaCT/"

gene = 'wt'
wt_fPaths = [
    "20211129_5dpf_xOG_aCtdT_wt_L07_002",
    "20211129_5dpf_xOG_aCtdT_wt_L07_004",
    "20211129_5dpf_xOG_aCtdT_wt_L09_002",
    "20211129_5dpf_xOG_aCtdT_wt_L09_004",
         ]
for fPath in wt_fPaths:
    cellCounter(viewer,dAnalysis,gene,fPath,'R')
    
    
gene = 'skor1a'
skor1a_fPaths = [
    "20211129_5dpf_xOG_aCtdT_skor1aF0_L07_002",
    "20211129_5dpf_xOG_aCtdT_skor1aF0_L07_004",
    "20211129_5dpf_xOG_aCtdT_skor1aF0_L08_002", #from fragment analysis: this is wt
    "20211129_5dpf_xOG_aCtdT_skor1aF0_L08_004", #from fragment analysis: this is wt
    "20211129_5dpf_xOG_aCtdT_skor1aF0_L09_002",
    "20211129_5dpf_xOG_aCtdT_skor1aF0_L09_004",
    "20211129_5dpf_xOG_aCtdT_skor1aF0_L10_002",
    "20211129_5dpf_xOG_aCtdT_skor1aF0_L11_002",
    "20211129_5dpf_xOG_aCtdT_skor1aF0_L12_002",
    "20211129_5dpf_xOG_aCtdT_skor1aF0_L12_004",
    "20211129_5dpf_xOG_aCtdT_skor1aF0_L13_002",
    "20211129_5dpf_xOG_aCtdT_skor1aF0_L13_004",
    "20211129_5dpf_xOG_aCtdT_skor1aF0_L14_002",
    "20211129_5dpf_xOG_aCtdT_skor1aF0_L15_002",
    "20211129_5dpf_xOG_aCtdT_skor1aF0_L16_002",
    "20211129_5dpf_xOG_aCtdT_skor1aF0_L17_002",
    "20211129_5dpf_xOG_aCtdT_skor1aF0_L17_004",
    "20211129_5dpf_xOG_aCtdT_skor1aF0_L18_002",
    "20211129_5dpf_xOG_aCtdT_skor1aF0_L18_004",
         ]
for fPath in skor1a_fPaths:
    cellCounter(viewer,dAnalysis,gene,fPath,'R')
        


In [ ]:
# viewer = napari.Viewer()

In [ ]:
# divide images into 1000 bins
mH, mB = np.histogram(viewer.layers['M_mip'].data.flatten(), bins = 1000)
lH, lB = np.histogram(viewer.layers['L_mip'].data.flatten(), bins = 1000)


fH, axH = plt.subplots(figsize=(8,8))
axH.stairs(lH,lB, fill=True, color = 'm', alpha = 0.5)
axH.stairs(mH,mB, fill=True, color = 'g', alpha = 0.5)

In [ ]:
viewer.close()

In [ ]:
viewer

```python
# change keybind for default actions:
@viewer.bind_key('F1', overwrite=True)
def eraserMode(viewer):
    viewer.layers.selection.active.mode = 'fill'
    viewer.layers.selection.active.mode = 'paint'
    viewer.layers.selection.active.mode = 'erase'
    viewer.layers.selection.active.mode = 'pick'
    viewer.layers.selection.active.mode = 'pan_zoom'

# napari native functions for other actions

@register_label_action(
    trans._(
        "Set the currently selected label to the largest used label plus one."
    ),
)
def new_label(layer):
    """Set the currently selected label to the largest used label plus one."""
    layer.selected_label = layer.data.max() + 1


@register_label_action(
    trans._("Decrease the currently selected label by one."),
)
def decrease_label_id(layer):
    layer.selected_label -= 1


@register_label_action(
    trans._("Increase the currently selected label by one."),
)
def increase_label_id(layer):
    layer.selected_label += 1

```

In [ ]:
viewer.layers['Seg'].data = dat['masks']

In [ ]:
dat = np.load("/Users/angueyraaristjm/Documents/LiImaging/A1R/zf_partial/20201009_CRtbx2a/20201009_i04c_L_seg.npy", allow_pickle=True).item()
import cellpose

In [ ]:
# dat['flows'];
t = cellpose.plot.mask_rgb(dat['masks'], colors=None)
# imgplot = plt.imshow(dat['flow'])
# dat

In [ ]:
imgplot = plt.imshow(t)

### Example notebook for cellpose [here](https://nbviewer.org/github/MouseLand/cellpose/blob/master/notebooks/run_cellpose.ipynb)
```python
# RUN CELLPOSE

from cellpose import models, io

# DEFINE CELLPOSE MODEL
# model_type='cyto' or model_type='nuclei'
model = models.Cellpose(gpu=False, model_type='cyto')

# define CHANNELS to run segementation on
# grayscale=0, R=1, G=2, B=3
# channels = [cytoplasm, nucleus]
# if NUCLEUS channel does not exist, set the second channel to 0
# channels = [0,0]
# IF ALL YOUR IMAGES ARE THE SAME TYPE, you can give a list with 2 elements
# channels = [0,0] # IF YOU HAVE GRAYSCALE
# channels = [2,3] # IF YOU HAVE G=cytoplasm and B=nucleus
# channels = [2,1] # IF YOU HAVE G=cytoplasm and R=nucleus

# or if you have different types of channels in each image
channels = [[2,3], [0,0], [0,0]]

# if diameter is set to None, the size of the cells is estimated on a per image basis
# you can set the average cell `diameter` in pixels yourself (recommended) 
# diameter can be a list or a single number for all images

# you can run all in a list e.g.
# >>> imgs = [io.imread(filename) in for filename in files]
# >>> masks, flows, styles, diams = model.eval(imgs, diameter=None, channels=channels)
# >>> io.masks_flows_to_seg(imgs, masks, flows, diams, files, channels)
# >>> io.save_to_png(imgs, masks, flows, files)

# or in a loop
for chan, filename in zip(channels, files):
    img = io.imread(filename)
    masks, flows, styles, diams = model.eval(img, diameter=None, channels=chan)

    # save results so you can load in gui
    io.masks_flows_to_seg(img, masks, flows, diams, filename, chan)

    # save results as png
    io.save_to_png(img, masks, flows, filename)
```

> flows[0] is XY flow in RGB  
> flows[1] is the cell probability in range 0-255 instead of 0.0 to 1.0  
> flows[2] is Z flow in range 0-255 (if it exists, otherwise zeros)  
> flows[3] is [dY, dX, cellprob] (or [dZ, dY, dX, cellprob] for 3D)  
> flows[4] is pixel destinations (for internal use)